# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7fbfb451ff10>
├── 'a' --> tensor([[-0.3292, -1.4743,  0.4442],
│                   [-1.9950,  0.1498, -0.9798]])
└── 'x' --> <FastTreeValue 0x7fbfb451feb0>
    └── 'c' --> tensor([[ 1.4402, -1.5920,  1.2373, -1.3680],
                        [-0.4187,  1.3880, -0.8032,  0.4080],
                        [ 0.3885, -0.9498,  0.3135, -1.0822]])

In [4]:
t.a

tensor([[-0.3292, -1.4743,  0.4442],
        [-1.9950,  0.1498, -0.9798]])

In [5]:
%timeit t.a

62.1 ns ± 0.0257 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7fbfb451ff10>
├── 'a' --> tensor([[-0.0393, -1.0967, -0.2856],
│                   [-0.3881,  0.8480, -1.3914]])
└── 'x' --> <FastTreeValue 0x7fbfb451feb0>
    └── 'c' --> tensor([[ 1.4402, -1.5920,  1.2373, -1.3680],
                        [-0.4187,  1.3880, -0.8032,  0.4080],
                        [ 0.3885, -0.9498,  0.3135, -1.0822]])

In [7]:
%timeit t.a = new_value

65.8 ns ± 0.043 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[-0.3292, -1.4743,  0.4442],
               [-1.9950,  0.1498, -0.9798]]),
    x: Batch(
           c: tensor([[ 1.4402, -1.5920,  1.2373, -1.3680],
                      [-0.4187,  1.3880, -0.8032,  0.4080],
                      [ 0.3885, -0.9498,  0.3135, -1.0822]]),
       ),
)

In [10]:
b.a

tensor([[-0.3292, -1.4743,  0.4442],
        [-1.9950,  0.1498, -0.9798]])

In [11]:
%timeit b.a

51.3 ns ± 0.0464 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[ 2.2731, -0.2978,  0.9771],
               [-0.4943,  0.1503, -0.4677]]),
    x: Batch(
           c: tensor([[ 1.4402, -1.5920,  1.2373, -1.3680],
                      [-0.4187,  1.3880, -0.8032,  0.4080],
                      [ 0.3885, -0.9498,  0.3135, -1.0822]]),
       ),
)

In [13]:
%timeit b.a = new_value

488 ns ± 0.214 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

923 ns ± 0.804 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

11.6 µs ± 15 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

139 µs ± 611 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

142 µs ± 548 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7fbf10e0c9a0>
├── 'a' --> tensor([[[-0.3292, -1.4743,  0.4442],
│                    [-1.9950,  0.1498, -0.9798]],
│           
│                   [[-0.3292, -1.4743,  0.4442],
│                    [-1.9950,  0.1498, -0.9798]],
│           
│                   [[-0.3292, -1.4743,  0.4442],
│                    [-1.9950,  0.1498, -0.9798]],
│           
│                   [[-0.3292, -1.4743,  0.4442],
│                    [-1.9950,  0.1498, -0.9798]],
│           
│                   [[-0.3292, -1.4743,  0.4442],
│                    [-1.9950,  0.1498, -0.9798]],
│           
│                   [[-0.3292, -1.4743,  0.4442],
│                    [-1.9950,  0.1498, -0.9798]],
│           
│                   [[-0.3292, -1.4743,  0.4442],
│                    [-1.9950,  0.1498, -0.9798]],
│           
│                   [[-0.3292, -1.4743,  0.4442],
│                    [-1.9950,  0.1498, -0.9798]]])
└── 'x' --> <FastTreeValue 0x7fbf10e15280>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

32.9 µs ± 31.3 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7fbf0a1ccdf0>
├── 'a' --> tensor([[-0.3292, -1.4743,  0.4442],
│                   [-1.9950,  0.1498, -0.9798],
│                   [-0.3292, -1.4743,  0.4442],
│                   [-1.9950,  0.1498, -0.9798],
│                   [-0.3292, -1.4743,  0.4442],
│                   [-1.9950,  0.1498, -0.9798],
│                   [-0.3292, -1.4743,  0.4442],
│                   [-1.9950,  0.1498, -0.9798],
│                   [-0.3292, -1.4743,  0.4442],
│                   [-1.9950,  0.1498, -0.9798],
│                   [-0.3292, -1.4743,  0.4442],
│                   [-1.9950,  0.1498, -0.9798],
│                   [-0.3292, -1.4743,  0.4442],
│                   [-1.9950,  0.1498, -0.9798],
│                   [-0.3292, -1.4743,  0.4442],
│                   [-1.9950,  0.1498, -0.9798]])
└── 'x' --> <FastTreeValue 0x7fbf0a1cc2e0>
    └── 'c' --> tensor([[ 1.4402, -1.5920,  1.2373, -1.3680],
                        [-0.4187,  1.3880, -0.8032,  0.4080],
                 

In [23]:
%timeit t_cat(trees)

30.8 µs ± 73.9 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

59.3 µs ± 182 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    a: tensor([[[-0.3292, -1.4743,  0.4442],
                [-1.9950,  0.1498, -0.9798]],
       
               [[-0.3292, -1.4743,  0.4442],
                [-1.9950,  0.1498, -0.9798]],
       
               [[-0.3292, -1.4743,  0.4442],
                [-1.9950,  0.1498, -0.9798]],
       
               [[-0.3292, -1.4743,  0.4442],
                [-1.9950,  0.1498, -0.9798]],
       
               [[-0.3292, -1.4743,  0.4442],
                [-1.9950,  0.1498, -0.9798]],
       
               [[-0.3292, -1.4743,  0.4442],
                [-1.9950,  0.1498, -0.9798]],
       
               [[-0.3292, -1.4743,  0.4442],
                [-1.9950,  0.1498, -0.9798]],
       
               [[-0.3292, -1.4743,  0.4442],
                [-1.9950,  0.1498, -0.9798]]]),
    x: Batch(
           c: tensor([[[ 1.4402, -1.5920,  1.2373, -1.3680],
                       [-0.4187,  1.3880, -0.8032,  0.4080],
                       [ 0.3885, -0.9498,  0.3135, -1.0822]],
         

In [26]:
%timeit Batch.stack(batches)

78.3 µs ± 257 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    a: tensor([[-0.3292, -1.4743,  0.4442],
               [-1.9950,  0.1498, -0.9798],
               [-0.3292, -1.4743,  0.4442],
               [-1.9950,  0.1498, -0.9798],
               [-0.3292, -1.4743,  0.4442],
               [-1.9950,  0.1498, -0.9798],
               [-0.3292, -1.4743,  0.4442],
               [-1.9950,  0.1498, -0.9798],
               [-0.3292, -1.4743,  0.4442],
               [-1.9950,  0.1498, -0.9798],
               [-0.3292, -1.4743,  0.4442],
               [-1.9950,  0.1498, -0.9798],
               [-0.3292, -1.4743,  0.4442],
               [-1.9950,  0.1498, -0.9798],
               [-0.3292, -1.4743,  0.4442],
               [-1.9950,  0.1498, -0.9798]]),
    x: Batch(
           c: tensor([[ 1.4402, -1.5920,  1.2373, -1.3680],
                      [-0.4187,  1.3880, -0.8032,  0.4080],
                      [ 0.3885, -0.9498,  0.3135, -1.0822],
                      [ 1.4402, -1.5920,  1.2373, -1.3680],
                      [-0.4187,  

In [28]:
%timeit Batch.cat(batches)

143 µs ± 399 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

312 µs ± 1.22 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
